<a href="https://colab.research.google.com/github/Rain1618/Hide-It/blob/main/dataset_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [42]:
df = pd.read_csv('/content/all_reddit_data - all_reddit_data.csv')
df.head()

,Unnamed: 0,title,selftext,subreddit,trigger,1,correct trigger
0,0,Could someone send me a message?,I'm having a really hard time and could use so...,alcoholism,addiction,1.0,safe
1,1,A rock bottom story,NaN,alcoholism,addiction,1.0,safe
2,2,Does this video infuriate anyone else or is it...,"Hi everyone. It's really nice to be here, I ju...",alcoholism,addiction,0.0,NaN
3,3,How do I feel joy when I'm not drunk?,I have a fantastic life. I have a beautiful wi...,alcoholism,addiction,0.0,NaN
4,4,What to do,I just hit 5 months sober which is still mind ...,alcoholism,addiction,NaN,NaN


In [43]:
df['trigger'] = np.where(df['1'] == 1, df['correct trigger'], df['trigger'])
df = df.drop(['Unnamed: 0', '1', 'correct trigger'], axis = 1)
df = df.replace(np.nan,'',regex=True)

df['text'] = df['title'] + ' ' + df['selftext']
df = df.drop(['title', 'selftext'], axis = 1)

df.head()

,subreddit,trigger,text
0,alcoholism,safe,Could someone send me a message? I'm having a ...
1,alcoholism,safe,A rock bottom story
2,alcoholism,addiction,Does this video infuriate anyone else or is it...
3,alcoholism,addiction,How do I feel joy when I'm not drunk? I have a...
4,alcoholism,addiction,What to do I just hit 5 months sober which is ...


In [12]:
df['subreddit'].unique()

array(['alcoholism', 'addiction', 'OpiatesRecovery', 'stopdrinking',
       'stopsmoking', 'StopSpeeding', 'domesticviolence',
       'NarcissisticAbuse', 'TrueNarcissisticAbuse', 'abusesurvivors',
       'abusiverelationships', 'emotionalabuse', 'eatingdisorder',
       'AnorexiaNervosa', 'bulimia', 'eating_disorders',
       'BingeEatingDisorder', 'EDAnonymous', 'CasualConversation',
       'UpliftingNews', 'BenignExistence', 'DoesAnybodyElse',
       'YouShouldKnow', 'CongratsLikeImFive', 'sexualassault',
       'adultsurvivors', 'rape', 'rapecounseling', 'Molested',
       'SexualAssaultSurvivor', 'DFSASurvivors', 'SuicideWatch',
       'TimeToGo', 'SuicideBereavement', 'StopSelfHarm', 'AdultSelfHarm',
       'selfharmteens'], dtype=object)

In [44]:
#Remove self harm posts and replace with suicide ones --> make sure suicide doesn't overlap
self_harm_df = df.loc[df['subreddit'].isin(['StopSelfHarm', 'AdultSelfHarm',
       'selfharmteens'])]

df.drop(df.loc[df['subreddit'].isin(['StopSelfHarm', 'AdultSelfHarm',
       'selfharmteens'])].index, inplace=True)

print(len(self_harm_df))
print(len(df))
self_harm_df.head()

2937
31752


,subreddit,trigger,text
31752,StopSelfHarm,suicide,Why is it so difficult for people to react “no...
31753,StopSelfHarm,suicide,My personal line of thinking when it comes to ...
31754,StopSelfHarm,suicide,Helpp Okay I have a scar and I think it's infe...
31755,StopSelfHarm,suicide,475 days!!
31756,StopSelfHarm,suicide,Does this count? Is it something to be worried...


In [45]:
#Get the suicide posts 

suicide_df = pd.read_csv('/content/reddit_depression_suicidewatch.csv')
print(len(suicide_df))
suicide_df.head()

20363


,text,label
0,I recently went through a breakup and she said...,depression
1,"I do not know how to navigate these feelings, ...",depression
2,"So I have been with my bf for 5 months , and h...",depression
3,I am so exhausted of this. Just when I think I...,SuicideWatch
4,I have been severly bullied since i was 5 till...,depression


In [46]:
suicide_df.drop(suicide_df.loc[suicide_df['label']== 'depression'].index, inplace=True)
print(len(suicide_df))
suicide_df.head()

9992


,text,label
3,I am so exhausted of this. Just when I think I...,SuicideWatch
5,I am 20 year old with some good friends but I ...,SuicideWatch
8,it is looming around the corner again. It alwa...,SuicideWatch
9,there is.....foodAnd other things I will be ju...,SuicideWatch
13,I am on zoloft and focalin and it is changed m...,SuicideWatch


In [47]:
#Merge two datasets

subset_suicide_df = suicide_df.iloc[:3300]
subset_suicide_df = subset_suicide_df.rename({'label': 'subreddit'}, axis=1)
subset_suicide_df['trigger'] = 'suicide'
subset_suicide_df.head()

,text,subreddit,trigger
3,I am so exhausted of this. Just when I think I...,SuicideWatch,suicide
5,I am 20 year old with some good friends but I ...,SuicideWatch,suicide
8,it is looming around the corner again. It alwa...,SuicideWatch,suicide
9,there is.....foodAnd other things I will be ju...,SuicideWatch,suicide
13,I am on zoloft and focalin and it is changed m...,SuicideWatch,suicide


In [48]:
merged_df = pd.concat([df, subset_suicide_df], ignore_index=True)
print(len(merged_df))
# merged_df.sort_values('text', inplace=True)
merged_df.drop_duplicates(subset='text', keep='first', inplace=True)
len(merged_df)

35052


34799

In [49]:
merged_df

,subreddit,trigger,text
0,alcoholism,safe,Could someone send me a message? I'm having a ...
1,alcoholism,safe,A rock bottom story
2,alcoholism,addiction,Does this video infuriate anyone else or is it...
3,alcoholism,addiction,How do I feel joy when I'm not drunk? I have a...
4,alcoholism,addiction,What to do I just hit 5 months sober which is ...
...,...,...,...
35047,SuicideWatch,suicide,I have tried for years to get better. I do not...
35048,SuicideWatch,suicide,For the last couple of weeks I have been havin...
35049,SuicideWatch,suicide,i am tired. i do not want to be here anymore. ...
35050,SuicideWatch,suicide,I want to die. I do not feel like living anymo...


In [50]:
merged_df['trigger'].value_counts()

suicide                              6276
addiction                            5905
domestic violence                    5835
safe                                 5746
eating disorders                     5693
sexual assault                       5319
                                        5
domestic violence,sexual violence       2
sexual assault,domestic violence        2
abuse                                   2
suicide, domestic violence              1
domestic violence,suicide               1
addiction, eating disorders             1
sexual violence, abuse                  1
sexual assault, domestic violence       1
domestic violence, sexual assault       1
french?                                 1
suicide, addiction                      1
addiction,eating disorders              1
eating disorder                         1
addiction,suicide                       1
addiction,domestic violence             1
eating disorders, sexual assault        1
sexual violence                   

In [51]:
#Relabelling
merged_df = merged_df.replace('domestic violence','abuse',regex=True)
merged_df = merged_df.replace('sexual assault','sexual violence',regex=True)
merged_df = merged_df.replace('eating disorders','eating disorder',regex=True)
merged_df['trigger'].value_counts()

suicide                             6276
addiction                           5905
abuse                               5837
safe                                5746
eating disorder                     5694
sexual violence                     5320
                                       5
sexual violence,abuse                  2
sexual violence, abuse                 2
abuse,sexual violence                  2
french?                                1
abuse, sexual violence                 1
suicide, abuse                         1
abuse,suicide                          1
addiction, eating disorder             1
suicide, addiction                     1
addiction,eating disorder              1
addiction,suicide                      1
addiction,abuse                        1
eating disorder, sexual violence       1
Name: trigger, dtype: int64

In [52]:
merged_df.head()

,subreddit,trigger,text
0,alcoholism,safe,Could someone send me a message? I'm having a ...
1,alcoholism,safe,A rock bottom story
2,alcoholism,addiction,Does this video infuriate anyone else or is it...
3,alcoholism,addiction,How do I feel joy when I'm not drunk? I have a...
4,alcoholism,addiction,What to do I just hit 5 months sober which is ...


In [53]:
#Text preprocessing
import nltk
nltk.download('punkt')

merged_df['text'] = merged_df['text'].str.replace(r'[^\x00-\x7F]+', '', regex=True)
merged_df['text'] = merged_df['text'].str.lower()
merged_df['text'] = merged_df['text'].str.replace('https:\S+|www.\S+', '', case=False)

#Word level tokenize + remove stopwords
merged_df['token_text'] = merged_df.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
merged_df = merged_df.drop('text', axis = 1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
<ipython-input-53-760f738a35e3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  merged_df['text'] = merged_df['text'].str.replace('https:\S+|www.\S+', '', case=False)


In [54]:
print(len(merged_df))
merged_df

34799


,subreddit,trigger,token_text
0,alcoholism,safe,"[could, someone, send, me, a, message, ?, i, '..."
1,alcoholism,safe,"[a, rock, bottom, story]"
2,alcoholism,addiction,"[does, this, video, infuriate, anyone, else, o..."
3,alcoholism,addiction,"[how, do, i, feel, joy, when, i, 'm, not, drun..."
4,alcoholism,addiction,"[what, to, do, i, just, hit, 5, months, sober,..."
...,...,...,...
35047,SuicideWatch,suicide,"[i, have, tried, for, years, to, get, better, ..."
35048,SuicideWatch,suicide,"[for, the, last, couple, of, weeks, i, have, b..."
35049,SuicideWatch,suicide,"[i, am, tired, ., i, do, not, want, to, be, he..."
35050,SuicideWatch,suicide,"[i, want, to, die, ., i, do, not, feel, like, ..."


In [55]:
merged_df.to_csv('cleaned_HideIt_data.csv', index=False)